In [3]:
## LOAD MODEL
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:latest", temperature=0)

## TEST MESSAGING
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-03-20T14:11:59.046271Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1352118750, 'load_duration': 706286833, 'prompt_eval_count': 30, 'prompt_eval_duration': 268000000, 'eval_count': 27, 'eval_duration': 375000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a7f1e586-08d5-4bc3-adec-866decd9d772-0', usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57})

In [8]:
from collections import Counter
from datetime import datetime
import spacy
from textblob import TextBlob

class TripMemory:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
        self.past_trips = []
        self.user_preferences = {
            'activities': Counter(),
            'sentiment_by_activity': {},
            'location_types': Counter(),
            'travel_pace': Counter(),
            'interests': Counter(),
            'social_style': Counter(),
            'comfort_preferences': Counter(),
            'negative_experiences': [],
            'positive_experiences': [],
            'budget_pattern': [],
            'seasonal_preference': Counter()
        }
        self.user_name = ""

    def get_valid_input(self, prompt, input_type="text", allow_empty=False):
        while True:
            user_input = input(prompt).strip()
            if not allow_empty and not user_input:
                print("I didn't catch that. Could you please provide an answer?")
                continue
            if input_type == "number":
                try:
                    return float(user_input)
                except ValueError:
                    print("Please enter a valid number.")
            else:
                return user_input

    def analyze_text(self, text, context):
        doc = self.nlp(text.lower())
        blob = TextBlob(text)
        sentiment = blob.sentiment.polarity

        activities = []
        locations = []
        descriptors = []

        for sent in doc.sents:
            for token in sent:
                if token.dep_ == "ROOT" and token.pos_ == "VERB":
                    activity = token.text
                    activity_phrase = ' '.join([t.text for t in token.subtree])
                    activities.append((activity, activity_phrase, sentiment))

                if token.ent_type_ in ["GPE", "LOC"]:
                    locations.append(token.text)

                if token.pos_ == "ADJ":
                    descriptors.append(token.text)

        self.update_preferences(activities, locations, descriptors, sentiment, context)

    def update_preferences(self, activities, locations, descriptors, sentiment, context):
        for activity, phrase, sent in activities:
            self.user_preferences['activities'][activity] += 1
            if activity not in self.user_preferences['sentiment_by_activity']:
                self.user_preferences['sentiment_by_activity'][activity] = []
            self.user_preferences['sentiment_by_activity'][activity].append(sent)

        pace_indicators = {
            'relaxed': ['relaxing', 'peaceful', 'quiet', 'slow'],
            'busy': ['busy', 'exciting', 'packed', 'full'],
            'adventure': ['adventurous', 'thrilling', 'challenging']
        }

        for descriptor in descriptors:
            for pace, indicators in pace_indicators.items():
                if descriptor in indicators:
                    self.user_preferences['travel_pace'][pace] += 1

        if sentiment > 0:
            self.user_preferences['positive_experiences'].append(context)
        elif sentiment < 0:
            self.user_preferences['negative_experiences'].append(context)

    def collect_past_trips(self):
        print("Hi there! 👋 I'm your personal travel advisor.")
        self.user_name = self.get_valid_input("What's your name? ")
        print(f"\nNice to meet you, {self.user_name}! I'd love to learn about your travel experiences.")
        print("Please tell me about at least 3 of your past trips.")
        
        while len(self.past_trips) < 3:
            print(f"\n✈️ Let's talk about trip #{len(self.past_trips) + 1}!")
            
            trip = {}
            destination = self.get_valid_input("Where did you go? ")
            trip['destination'] = destination
            
            print(f"\nTell me about your experience in {destination}!")
            experience = self.get_valid_input("What was your typical day like there? ")
            self.analyze_text(experience, 'daily_experience')
            
            highlights = self.get_valid_input("What were the highlights of your trip? ")
            self.analyze_text(highlights, 'highlights')
            
            challenges = self.get_valid_input("Did you face any challenges or disappointments? ")
            self.analyze_text(challenges, 'challenges')
            
            local_interaction = self.get_valid_input("How did you interact with local culture and people? ")
            self.analyze_text(local_interaction, 'cultural_interaction')
            
            comfort = self.get_valid_input("How would you describe your comfort level during the trip? ")
            self.analyze_text(comfort, 'comfort')
            
            budget = self.get_valid_input("How would you describe the cost of this trip? (budget/moderate/luxury): ")
            trip['budget'] = budget
            self.user_preferences['budget_pattern'].append(budget)
            
            season = self.get_valid_input("What season did you travel in? (spring/summer/fall/winter): ")
            trip['season'] = season
            self.user_preferences['seasonal_preference'][season] += 1
            
            trip.update({
                'experience': experience,
                'highlights': highlights,
                'challenges': challenges,
                'local_interaction': local_interaction,
                'comfort': comfort
            })
            
            self.past_trips.append(trip)
            
            if len(self.past_trips) < 3:
                print(f"\nFascinating! Let's hear about {3 - len(self.past_trips)} more trip{'s' if 3 - len(self.past_trips) > 1 else ''}!")
        
        self.summarize_preferences()
        return self.user_preferences, self.user_name

    def summarize_preferences(self):
        print(f"\n📊 Based on analyzing your travel experiences, {self.user_name}, I've learned that:")
        self.summarize_travel_style()
        self.summarize_activity_preferences()
        self.summarize_comfort_preferences()
        self.provide_unique_insights()

    def summarize_travel_style(self):
        dominant_pace = self.user_preferences['travel_pace'].most_common(1)
        if dominant_pace:
            print(f"\n🎭 Travel Style:")
            print(f"You seem to prefer a {dominant_pace[0][0]} pace when traveling")

    def summarize_activity_preferences(self):
        print("\n🎯 Activity Preferences:")
        for activity, count in self.user_preferences['activities'].most_common(3):
            sentiments = self.user_preferences['sentiment_by_activity'].get(activity, [])
            avg_sentiment = sum(sentiments) / len(sentiments) if sentiments else 0
            
            if avg_sentiment > 0.3:
                emotion = "love"
            elif avg_sentiment > 0:
                emotion = "enjoy"
            else:
                emotion = "have mixed feelings about"
                
            print(f"- You seem to {emotion} {activity}")

    def summarize_comfort_preferences(self):
        print("\n🛋️ Comfort Preferences:")
        budget_pattern = Counter(self.user_preferences['budget_pattern'])
        dominant_budget = budget_pattern.most_common(1)[0][0]
        print(f"- You typically prefer {dominant_budget} level accommodations")

    def provide_unique_insights(self):
        print("\n💡 Unique Insights:")
        
        positive_words = [word.text for exp in self.user_preferences['positive_experiences'] 
                         for word in self.nlp(exp) if word.pos_ == "ADJ"]
        if positive_words:
            common_positive = Counter(positive_words).most_common(1)[0][0]
            print(f"- You particularly value {common_positive} experiences")

        comfort_score = sum(1 for exp in self.user_preferences['comfort_preferences'] 
                          if any(word in exp.lower() for word in ['comfortable', 'luxury', 'easy']))
        adventure_score = sum(1 for exp in self.user_preferences['activities'] 
                            if any(word in exp.lower() for word in ['adventure', 'challenge', 'explore']))
        
        if comfort_score > adventure_score:
            print("- You tend to prioritize comfort while still enjoying new experiences")
        elif adventure_score > comfort_score:
            print("- You're willing to step out of your comfort zone for unique experiences")

ModuleNotFoundError: No module named 'en_core_web_sm'

In [5]:
from datetime import datetime

class TripAdvisor:
    def __init__(self, user_preferences, user_name):
        self.user_preferences = user_preferences
        self.user_name = user_name
        self.destinations = {
            'Bali': {
                'activities': ['explore', 'swim', 'relax', 'surf'],
                'climate': 'tropical',
                'budget_level': ['moderate', 'luxury'],
                'best_seasons': ['spring', 'summer'],
                'travel_styles': ['relaxed', 'cultural'],
                'min_budget': 1500,
                'activities_list': [
                    '🏖️ Premium beach clubs in Seminyak',
                    '🏄‍♂️ Surfing lessons in Canggu',
                    '🕉️ Temple visits in Ubud',
                    '🚶‍♂️ Rice terrace walks',
                    '🏊‍♂️ Snorkeling in Nusa Penida'
                ],
                'description': 'A perfect blend of culture and relaxation'
            },
            'Japan': {
                'activities': ['explore', 'eat', 'walk', 'shop'],
                'climate': 'moderate',
                'budget_level': ['moderate', 'luxury'],
                'best_seasons': ['spring', 'fall'],
                'travel_styles': ['busy', 'cultural'],
                'min_budget': 3000,
                'activities_list': [
                    '🏯 Historic temples and shrines',
                    '🍜 Food tours and cooking classes',
                    '🗻 Mount Fuji hiking',
                    '🚅 Bullet train experiences',
                    '🌸 Cherry blossom viewing (spring)'
                ],
                'description': 'Perfect for cultural immersion and modern experiences'
            },
            'New Zealand': {
                'activities': ['hike', 'explore', 'drive', 'adventure'],
                'climate': 'moderate',
                'budget_level': ['moderate', 'luxury'],
                'best_seasons': ['spring', 'fall'],
                'travel_styles': ['adventure', 'nature'],
                'min_budget': 2500,
                'activities_list': [
                    '🥾 Tongariro Alpine Crossing',
                    '🚴‍♂️ Mountain biking in Rotorua',
                    '🚗 Scenic drives along the South Island',
                    '🦁 Wildlife watching in Fiordland',
                    '🪂 Adventure sports in Queenstown'
                ],
                'description': 'Ideal for nature lovers and adventure seekers'
            }
        }

    def get_valid_input(self, prompt, input_type="text"):
        while True:
            user_input = input(prompt).strip()
            if not user_input:
                print("I didn't catch that. Could you please provide an answer?")
                continue
            if input_type == "number":
                try:
                    return float(user_input)
                except ValueError:
                    print("Please enter a valid number.")
            else:
                return user_input

    def validate_date(self, date_str):
        try:
            date = datetime.strptime(date_str, '%m/%Y')
            if date < datetime.now():
                return False
            return True
        except ValueError:
            return False

    def get_travel_details(self):
        print(f"\nNow, let's plan your next adventure, {self.user_name}! 🌎")
        
        while True:
            when = self.get_valid_input("When are you thinking of traveling? (MM/YYYY): ")
            if self.validate_date(when):
                break
            print("Please enter a future date in the format MM/YYYY (e.g., 12/2024)")
        
        while True:
            budget = self.get_valid_input("What's your budget for this trip? (USD): ", "number")
            if budget > 0:
                break
            print("Please enter a valid budget amount greater than 0")
        
        while True:
            companions = self.get_valid_input("Who will you be traveling with? (solo/family/friends/partner): ").lower()
            if companions in ['solo', 'family', 'friends', 'partner']:
                break
            print("Please select one of: solo, family, friends, or partner")
        
        while True:
            duration = self.get_valid_input("How many days are you planning to stay? ", "number")
            if 1 <= duration <= 90:
                break
            print("Please enter a reasonable duration between 1 and 90 days")
        
        return {
            'when': when,
            'budget': budget,
            'companions': companions,
            'duration': duration
        }

    def calculate_destination_score(self, destination, attributes, travel_details):
        score = 0
        
        # Match activities
        for activity in attributes['activities']:
            score += self.user_preferences['activities'][activity]
        
        # Match travel style
        for style in attributes['travel_styles']:
            if style in self.user_preferences['travel_pace']:
                score += self.user_preferences['travel_pace'][style]
        
        # Budget compatibility
        if travel_details['budget'] >= attributes['min_budget']:
            score += 2
        
        # Season compatibility
        travel_date = datetime.strptime(travel_details['when'], '%m/%Y')
        season = self.get_season(travel_date.month)
        if season in attributes['best_seasons']:
            score += 1
        
        return score

    def get_season(self, month):
        if month in [3, 4, 5]:
            return 'spring'
        elif month in [6, 7, 8]:
            return 'summer'
        elif month in [9, 10, 11]:
            return 'fall'
        else:
            return 'winter'

    def recommend_destination(self, travel_details):
        print("\n🔍 Analyzing your travel preferences and requirements...")
        
        destination_scores = {}
        for dest, attributes in self.destinations.items():
            score = self.calculate_destination_score(dest, attributes, travel_details)
            destination_scores[dest] = score

        top_destinations = sorted(destination_scores.items(), key=lambda x: x[1], reverse=True)[:2]
        
        print(f"\nBased on your travel style and preferences, {self.user_name}, I have two excellent recommendations for you!")
        
        for dest, score in top_destinations:
            self.present_recommendation(dest, travel_details)

    def present_recommendation(self, destination, travel_details):
        dest_info = self.destinations[destination]
        print(f"\n🌟 Recommended Destination: {destination}")
        print(f"\n{dest_info['description']}")
        
        print("\nWhy this matches your style:")
        matching_activities = set(dest_info['activities']) & set(self.user_preferences['activities'].keys())
        if matching_activities:
            print("- Aligns with your interest in:", ", ".join(matching_activities))
        
        if travel_details['budget'] >= dest_info['min_budget']:
            print(f"- Fits your budget of ${travel_details['budget']:,.2f}")
        
        print("\nRecommended activities:")
        for activity in dest_info['activities_list']:
            print(activity)
        
        print(f"\nBest time to visit: {', '.join(dest_info['best_seasons']).title()}")
        
        # Additional personalized tips
        if 'cultural' in dest_info['travel_styles']:
            print("\n💡 Cultural Tip: Consider learning a few basic local phrases to enhance your experience!")
        
        if dest_info['climate'] == 'tropical':
            print("🌡️ Weather Tip: Pack light, breathable clothing and don't forget sun protection!")

def main():
    # First part: Collect travel history and build memory
    memory = TripMemory()
    user_preferences, user_name = memory.collect_past_trips()
    
    # Second part: Get travel recommendations
    advisor = TripAdvisor(user_preferences, user_name)
    travel_details = advisor.get_travel_details()
    advisor.recommend_destination(travel_details)

if __name__ == "__main__":
    main()

NameError: name 'TripMemory' is not defined

In [11]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
